# Agent-Based Model developed by Ejsmond 2018

Port of the simulation of [Ejsmond 2018](https://www.nature.com/articles/s41467-018-06821-x) from matlab to python. Original code: [https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-018-06821-x/MediaObjects/41467_2018_6821_MOESM4_ESM.txt](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-018-06821-x/MediaObjects/41467_2018_6821_MOESM4_ESM.txt).

In [1]:
import numpy as np
import pandas as pd

## Initialize Global Model Variables

In [2]:
rng = np.random.default_rng(); # initialize RNG

N = 1000  # population size
n_gen = 10600  # generation number
epitope_space_size = 1000 # "epitope" space size (1000*1000)

red_queen = True # toggles co-evolution of host-parasite -> 

dcl_ST = True # toggle ST analysis - only plotting !not clear!

# data for plotting is only saved every 10th generation, but instead of just going "generation%10", this solution was decided upon.
start_save = 2000  # start of saving data on host and pathogens
plotting_generations = range(start_save, n_gen + 1, 10)  # plotting of ST and haplotypes
plotting_index = 0  # index for tracking the plotting data



## Initialize Host Data

In [3]:
host_mu = 0.1 # host mutation rate
# possible step directions a host can take when mutating
host_mu_steps = np.matrix([
    [-1, -1, -1,  0, 0, 0,  1, 1, 1],
    [-1,  0,  1, -1, 0, 1, -1, 0, 1]
]) 

host_fitness_reproduction = 1.25 # fitness of reproducing hosts
host_fitness_birth = 0.25 # fitness of hosts at birth

# initialize positions of allele one in epitope space (x,y)
host_allele_1_start = rng.integers(low=1, high=epitope_space_size, size=(2, N)) 
# initialize positions of allele two in epitope space (x,y)
host_allele_2_start = rng.integers(low=1, high=epitope_space_size, size=(2, N))

host_allele_1 = host_allele_1_start
host_allele_2 = host_allele_2_start

# later used, defined here for global scope
# effective number of alleles, subsample from 100 !not entirely clear!
host_allele_count_effective = np.zeros((len(plotting_generations))) 

# record values over all the generations of the simulation (vector length == number of generations)
# number of dying hosts from each generation
hist_host_dying = np.zeros((n_gen))
# number of hosts not reproducing
hist_host_not_reproducing = np.zeros((n_gen))

# initial fitness for all hosts
host_fitness = np.zeros((N)) + host_fitness_birth
# tally for host brood
host_brood_count = np.zeros((N))
host_age = np.zeros((N))

# Supertype start positions (x,y) !not clear!
STs_1_2_start = np.matrix([range(1,N+1), range(N+1, 2*N+1)])


## Initialize Parasite Data

In [4]:
pat_count = 10  # number of pathogen species
pat_mu = 1
pat_mu_steps = np.matrix([-1, 0, 1])
pat_added = 0.01  # fraction of pathogens from initial population that is added every generation to the pool from which recruits are drawn

# pathogen haplotype, as x,y coordinate in the epitope space
pat_haplotype_start = rng.integers(low=1, high=epitope_space_size, size=(2, N, pat_count))

# record pathogen properties in plotting_generations increments
g4hist_pat_haplotype_count = np.zeros((pat_count,len(plotting_generations)))
g4hist_pat_haplotype = np.zeros((N,len(plotting_generations),2))


## Running the Simulation

In [23]:
for iteration in range(1,n_gen + 1):
  # print iteration count for every 212th generation
  #if iteration % 212 == 0:
    # print(iteration)
  # save data for plotting every 10th generation. Just do generations % 10?
  if iteration in plotting_generations:
    plotting_index += 1
    # subsample 100 hosts alleles
    host_allele_subsample_index = rng.integers(low=1, high=N, size=(2,100))
    print(host_allele_subsample_index)
    break

    


[[688 888 888 770 220 154 299 782 316 654 216 330 400 721 470 283 730 145
  794 565 627  39 558 238 282 614 442 604 612 593 319 110 367 690 596 900
  410 868 968 708 413 372 415 593 226 552 374 436 554 946 781 990 318 789
  790 709 117  86 926 344 144  43 540 435 856 701 626 521 400 127 248  98
  866 496 472 645 628 190 865 354 748 240 722 458  86 278 388 855 859 384
  519 787 250  91 608 442 566 557 695  73]
 [827 199 325 727 953  29 440  51 607 670 717 586 485 566 204 113 160 284
  562 481  21 399 126  85 836 418 664 671 312 533 147 656 406  25 899 257
  857 886 213 100 860 862 908 863  15 296 767 128 159 707 862 670 326 224
  508 914 901   6 458 564 955 222 546 657  61 912 154  66 271 267 100 467
  105 455  66 911 914 769 105 972 535  91  71 353 397  13 125  76  53 681
  380  28 809 231 404 407 794 962 976 817]]
